In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# --- 1. Load and Preprocess the MNIST Dataset ---
print("Loading MNIST dataset...")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# --- 2. Reshape Data for the Model ---
# The ImageDataGenerator and Conv2D layers expect a 4D tensor (samples, height, width, channels)
print("Reshaping data...")
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")


# --- 3. Define the Model Architecture ---
print("Building the model...")
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation="relu", input_shape=(28,28,1)))
model.add(layers.Conv2D(64, (3,3), activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.Conv2D(128, (3,3), activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation="softmax"))

# --- 4. Compile the Model ---
print("Compiling the model...")
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

# --- 5. Create the Data Augmentation Generator ---
print("Creating data augmentation generator...")
datagen = ImageDataGenerator(
    rotation_range=10,      # randomly rotate images by 10 degrees
    zoom_range=0.1,         # Randomly zoom image by 10%
    width_shift_range=0.1,  # randomly shift images horizontally by 10%
    height_shift_range=0.1, # randomly shift images vertically by 10%
)

# --- 6. Train the Model Using the Generator ---
print("Starting model training with data augmentation...")
history = model.fit(datagen.flow(x_train, y_train, batch_size=1024),
                    epochs=20,
                    validation_data=(x_test, y_test), # Use the test set for validation
                    verbose=1)

# --- 7. Evaluate the Final Model ---
print("\nEvaluating the model on the test set...")
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"\nTest accuracy: {test_acc:.4f}")

# --- 8. Save the Trained Model ---
# You can now use this file in your main Sudoku solver script
model_filename = 'sudoku_digit_model.h5'
print(f"Saving trained model to {model_filename}...")
model.save(model_filename)
print("Model saved successfully!")



Loading MNIST dataset...
Reshaping data...
x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
Building the model...
Compiling the model...


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 10, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 786,122 (3.00 MB)

 Trainable params: 786,122 (3.00 MB)

 Non-trainable params: 0 (0.00 B)

Creating data augmentation generator...
Starting model training with data augmentation...
Epoch 1/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 38s 440ms/step - accuracy: 0.5187 - loss: 1.3831 - val_accuracy: 0.9692 - val_loss: 0.0929
Epoch 2/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 18s 309ms/step - accuracy: 0.9174 - loss: 0.2650 - val_accuracy: 0.9860 - val_loss: 0.0451
Epoch 3/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 18s 300ms/step - accuracy: 0.9512 - loss: 0.1640 - val_accuracy: 0.9878 - val_loss: 0.0364
Epoch 4/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 18s 302ms/step - accuracy: 0.9639 - loss: 0.1197 - val_accuracy: 0.9907 - val_loss: 0.0283
Epoch 5/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 18s 304ms/step - accuracy: 0.9684 - loss: 0.1021 - val_accuracy: 0.9922 - val_loss: 0.0238
Epoch 6/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 18s 305ms/step - accuracy: 0.9748 - loss: 0.0876 - val_accuracy: 0.9939 - val_loss: 0.0195
Epoch 7/20
59/59 ━━━━━━━━━━━━━━━━━━━━ 18s 303ms/step - accuracy: 0.9796 - loss: 0.0705 - val_accuracy: 0.9925 - val_loss: 0.0212
Epoch 8